In [ ]:
from image_prediction import predict_image

In [14]:
print(predict_image('images/dog.jpg'))

0.0005424591945484281


In [13]:
print(predict_image('images/aidog.jpeg'))

0.9999719858169556


In [15]:
print(predict_image('images/ai_1.jpeg'))

0.9997043013572693


In [16]:
print(predict_image('images/ai_2.jpeg'))

0.9992165565490723


In [ ]:
print(predict_image('images/real_1.jpg')) # model is wrong

0.7752610445022583


In [ ]:
print(predict_image('images/real_2.jpg')) # model is wrong

0.984261155128479


In [ ]:
print(predict_image('images/real_3.jpg'))

0.0005000822129659355


In [ ]:
print(predict_image('images/NYT_test.webp'))

0.9998764991760254


In [23]:
print(predict_image('images/NYT_test_2.webp'))

0.9998494386672974
